In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [14]:
file_path = 'connect4_dataset.csv'
dataset = pd.read_csv(file_path)

dataset.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.32,0.33,0.34,-1,-1.1,0.35,0.36,1.1,1.2,0.37
0,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,-1,-1,0,-1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,-1,1,1,1,1,1,1,-1,-1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,-1,-1,-1,1,1,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,-1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,1,1,-1,-1,-1,1,0


In [15]:
# Preprocess data
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

valid_indices = (y >= 0) & (y <= 6)
X = X[valid_indices]
y = y[valid_indices]

y = to_categorical(y, num_classes=7)

X = X.reshape(-1, 6, 7, 1)

X_flat = X.reshape(X.shape[0], -1)

In [16]:
# Build and Train Model
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

mlp_model = Sequential([
    Dense(128, activation='tanh', input_shape=(42,)),
    Dropout(0.5),
    Dense(64, activation='tanh'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

optimizer = Adam(learning_rate=0.0001)
mlp_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = mlp_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.3397 - loss: 1.8123 - val_accuracy: 0.7012 - val_loss: 1.0983
Epoch 2/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6848 - loss: 1.1982 - val_accuracy: 0.7013 - val_loss: 1.0743
Epoch 3/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6915 - loss: 1.1592 - val_accuracy: 0.7013 - val_loss: 1.0636
Epoch 4/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6986 - loss: 1.1263 - val_accuracy: 0.7013 - val_loss: 1.0553
Epoch 5/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7006 - loss: 1.1087 - val_accuracy: 0.7013 - val_loss: 1.0509
Epoch 6/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7007 - loss: 1.1022 - val_accuracy: 0.7013 - val_loss: 1.0473
Epoch 7/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7003 - loss: 1.0877 - val_accuracy: 0.7013 - val_loss: 1.0452
Epoch 8/15
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6998 - loss: 1.0867 - val_accurac

In [17]:
# Evaluate the MLP model on the test set
test_loss, test_accuracy = mlp_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6994 - loss: 1.0345
Test Accuracy: 0.70


In [18]:
mlp_model.save("connect4_mlp_model.h5")